# Setting Up Required Libraries

In [ ]:
import ee
import geemap
import geemap.colormaps as cm

In [ ]:
ee.Authenticate()
ee.Initialize(project='-----Insert Project ID Here-----')

# Study Area

In [ ]:
valencia = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level1') \
    .filter(ee.Filter.eq('ADM0_NAME', 'Spain')) \
    .filter(ee.Filter.eq('ADM1_NAME', 'Comunitat Valenciana'))

valencia = valencia.geometry()

Map = geemap.Map(height=600)
Map.centerObject(valencia, 8)

Map.addLayer(valencia, {}, 'Comunitat Valenciana')


# Precipitation Analysis in Comunitat Valenciana, Espana

In [ ]:
dataset = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filter(ee.Filter.date('2024-01-01', '2024-12-31'))

In [ ]:
precipitation = dataset.select('precipitation')

In [ ]:
# Calculate the mean monthly precipitation for 2024
monthly_precipitation = precipitation.mean().reduceRegion(**{
    'reducer': ee.Reducer.mean(),
    'geometry': valencia,
    'scale': 5000,
    'maxPixels': 1e13
}).get('precipitation')

In [ ]:
# Get the months (1-12)
months = ee.List.sequence(1, 12)

In [ ]:
# Function to calculate the mean precipitation for a specific month
def calculate_monthly_mean(month):
  monthly_image = precipitation.filter(ee.Filter.calendarRange(month, month, 'month')) \
      .mean() \
      .reduceRegion(**{
          'reducer': ee.Reducer.mean(),
          'geometry': valencia,
          'scale': 5000,
          'maxPixels': 1e13
      })
  return monthly_image.get('precipitation')

In [ ]:
monthly_means = months.map(calculate_monthly_mean)

# Convert the mean precipitation list to a client-side list
monthly_means_list = monthly_means.getInfo()

In [ ]:
# Create a list of month labels
month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Create a chart
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 5))
plt.bar(month_labels, monthly_means_list, color='aqua')
plt.xlabel('Month')
plt.ylabel('Precipitation (mm)')
plt.title('Daily Precipitation Average in Comunitat Valenciana, 2024')

import numpy as np
# Add trend line
z = np.polyfit(range(len(monthly_means_list)), monthly_means_list, 1)
p = np.poly1d(z)
plt.plot(month_labels, p(range(len(monthly_means_list))), "y--")
plt.savefig('Valencia Daily Precipitation Average per Month.png')
# Show the chart
plt.show()

# Radar Visualization

In [ ]:
before_radar = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(valencia) \
    .filterDate('2024-10-01', '2024-10-08') \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .select('VH') \
    .median() \
    .clip(valencia)

In [ ]:
after_radar = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(valencia) \
    .filterDate('2024-10-21', '2024-11-01') \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .select('VH') \
    .median() \
    .clip(valencia)

In [ ]:
# Define visualization parameters
vis_params = {
    'min': -25,
    'max': 0,
    'palette': ['black', 'white']
}

palette = cm.palettes.Blues_r

flood_vis = {
    'min': -25,
    'max': 0,
    'palette': palette
}

In [ ]:
flood_sar = after_radar.subtract(before_radar)

In [ ]:
Map.addLayer(before_radar, vis_params, 'SAR - Before', False)
Map.addLayer(after_radar, vis_params, 'SAR - After', False)
Map.addLayer(flood_sar, flood_vis, 'Flood - SAR')
Map

# Optical Visualization

In [ ]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

In [ ]:
sen2_before = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2024-10-01', '2024-10-08')
    .filterBounds(valencia)
    .mosaic()
)

In [ ]:
sen2_after = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2024-10-29', '2024-11-01')
    .filterBounds(valencia)
    .mosaic()
)

In [ ]:
Map.addLayer(sen2_before, {'bands': ['B8', 'B4', 'B3']}, 'Sentinel-2 Before', False)

Map.addLayer(sen2_after, {'bands': ['B8', 'B4', 'B3']}, 'Sentinel-2 After', False)

Map